In [ ]:
from google.colab import files
upload = files.upload()

In [ ]:
! unzip "tcpSyn-arpPoison Generalization".zip

Archive:  tcpSyn-arpPoison Generalization.zip
replace tcpSyn-arpPoison Generalization/arp-poison.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace tcpSyn-arpPoison Generalization/proc-tcp-syn.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import csv
import tensorflow as tf
import numpy as np
import ipaddress

In [ ]:
import re
def mac_to_int(mac):
    res = re.match('^((?:(?:[0-9a-f]{2}):){5}[0-9a-f]{2})$', mac.lower())
    if res is None:
        raise ValueError('invalid mac address')
    return int(res.group(0).replace(':', ''), 16)

In [ ]:
datafile = open('tcpSyn-arpPoison Generalization/proc-tcp-syn.csv','r')
csvr = csv.reader(datafile)
next(csvr)
data = []
label = []
for row in csvr:
  row[0] = mac_to_int(row[0])
  row[1] = mac_to_int(row[1])
  if row[2] == '':
    row[2] = 00000000
  else:
    row[2] = int(ipaddress.ip_address(row[2]))
  if row[3] == '':
    row[3] = 00000000
  else:
    row[3] = int(ipaddress.ip_address(row[3]))
  data.append(row[0:-1])
  label.append(row[-1])

In [ ]:
for i,r in enumerate(label):
  if r == 'benign':
    label[i] = 0
  else:
    label[i] = 1

In [ ]:
train_data = np.asarray(data[0 : int(len(data) * 0.8)])
valid_data = np.asarray(data[int(len(data) * 0.8) : int(len(data) * 0.9)])
test_data = np.asarray(data[int(len(data) * 0.9) : ])

train_label = np.asarray(label[0 : int(len(label) * 0.8)])
valid_label = np.asarray(label[int(len(label) * 0.8) : int(len(label) * 0.9)])
test_label = np.asarray(label[int(len(label) * 0.9) : ])

train_label = tf.keras.utils.to_categorical(train_label, 2)
valid_label = tf.keras.utils.to_categorical(valid_label, 2)
test_label = tf.keras.utils.to_categorical(test_label, 2)

In [ ]:
input_size = len(train_data[0])
output_size = len(set(label))
hidden_layer_size = 75
ACT_FN = 'relu'

model = tf.keras.Sequential([
          tf.keras.layers.Dense(30, input_dim=input_size, kernel_initializer='uniform', activation='relu'), # input layer

          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 1st hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 2nd hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 3nd hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 4nd hidden layer
          tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 30)                150       
                                                                 
 dense_31 (Dense)            (None, 75)                2325      
                                                                 
 dense_32 (Dense)            (None, 75)                5700      
                                                                 
 dense_33 (Dense)            (None, 75)                5700      
                                                                 
 dense_34 (Dense)            (None, 75)                5700      
                                                                 
 dense_35 (Dense)            (None, 2)                 152       
                                                                 
Total params: 19,727
Trainable params: 19,727
Non-trai

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import TensorBoard
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
max_epochs = 2
BATCH_SIZE = 2

In [ ]:
model.fit(train_data,
          train_label,
          # batch_size=BATCH_SIZE,
          epochs=max_epochs,
          # callbacks=tb,
          validation_data=(valid_data, valid_label),
          validation_steps=10,
          verbose=1
          )

Epoch 1/2
38424/38424 [==============================] - 97s 3ms/step - loss: 403.3133 - accuracy: 0.9029 - val_loss: 0.0994 - val_accuracy: 1.0000
Epoch 2/2
38424/38424 [==============================] - 96s 2ms/step - loss: 0.3110 - accuracy: 0.9064 - val_loss: 0.0937 - val_accuracy: 1.0000


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, test_label)

4803/4803 [==============================] - 8s 2ms/step - loss: 0.0937 - accuracy: 1.0000


In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 100.00%


In [ ]:
# 
#  TESTING MODEL ON AN ENTIRELY NEW DATASET
#

In [ ]:
datafile2 = open('tcpSyn-arpPoison Generalization/arp-poison.csv','r')
csvr2 = csv.reader(datafile2)
next(csvr2)
data2 = []
label2 = []
for row in csvr2:
  row[0] = mac_to_int(row[0])
  row[1] = mac_to_int(row[1])
  if row[2] == '':
    row[2] = 00000000
  else:
    row[2] = int(ipaddress.ip_address(row[2]))
  if row[3] == '':
    row[3] = 00000000
  else:
    row[3] = int(ipaddress.ip_address(row[3]))
  data2.append(row[0:-1])
  label2.append(int(row[-1]))

In [ ]:
for i,r in enumerate(label2):
  if r > 0:
    label2[i] = 1
  else:
    label2[i] = 0

In [ ]:
test_data2 = np.asarray(data2)
test_label2 = tf.keras.utils.to_categorical(label2, 2)

In [ ]:
test_loss2, test_accuracy2 = model.evaluate(test_data2, test_label2)

4188/4188 [==============================] - 7s 2ms/step - loss: 178209232519168.0000 - accuracy: 0.2514


In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss2, test_accuracy2*100.))

Test loss: 178209232519168.00. Test accuracy: 25.14%
